# 05 Clustering
Author: Murthadza bin Aznam <br>
Date: 9th August 2021<br>

This Notebook is written as part of the International Virtual Conference on Astrostatistics and Machine Learning (IVCASML2021) course.

---

## 0.0 GOAL
1. Determine the members of a cluster using HDBSCAN

---
---
# ! WARNING !
This interactive notebook is heavy. It will significantly slow down as the notebook progresses. I suspect it is because of the sheer volume of data that it needs to manage. Therefore, I have scatter a couple of `del` commands to delete unnecessary data. However, it is still heavy.

---
---

## 1 PREPARATION
### 1.1 PREPARATION: Importing packages

In [ ]:
import math
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import hdbscan
from sklearn.preprocessing import StandardScaler

### 1.2 PREPARATION: Set some plotting configurations

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)            # legend fontsize

%matplotlib inline

### 1.3 PREPARATION: Data Preparation
There are three different data available (Comment and uncomment accordingly):
1. NGC725
2. M67
3. NGC7789

The data is collected from Gaia Early Data Release 3

In [ ]:
DATADIR = "../dataset/"

#FILENAME = "gaiaedr3_180_NGC752"
#CLUSTER = "NGC752"

FILENAME = "gaiaedr3_150_M67"
CLUSTER = "M67"

#FILENAME = "gaiaedr3_45_NGC7789"
#CLUSTER = "NGC7789"

OUTPUTDIR = "../output/"

datafile = pd.read_csv(DATADIR + FILENAME + ".csv", delimiter=",")
datafile.head()

### 1.4 PREPARATION: Handling the missing values 

In [ ]:
datafile = datafile.dropna(subset=['pmra', 'pmdec', 'parallax']).reset_index()

To eliminate sources with high uncertainty while still retaining a fraction of sources down to G ∼ 21 mag, we need to select the errors in their G-mag must be less than 0.005. Calculate error of G ($|\sigma_G|$), $G_{BP}$ ($|\sigma_{BP}|$), and $G_{RP}$ ($|\sigma_{RP}|$).

$$\begin{array}
|\sigma_G| &= -\frac{2.5}{ln \ 10} \frac{\sigma_{F_G}}{F_G} \\
|\sigma_{BP}| &= -\frac{2.5}{ln \ 10} \frac{\sigma_{F_{BP}}}{F_{BP}} \\
|\sigma_{RP}| &= -\frac{2.5}{ln \ 10} \frac{\sigma_{F_{RP}}}{F_{RP}} \\
\end{array}$$

In [ ]:
datafile['e_Gmag'] = abs(-2.5*datafile['phot_g_mean_flux_error']/math.log(10)/datafile['phot_g_mean_flux'])
datafile['e_BPmag'] = abs(-2.5*datafile['phot_bp_mean_flux_error']/math.log(10)/datafile['phot_bp_mean_flux'])
datafile['e_RPmag'] = abs(-2.5*datafile['phot_rp_mean_flux_error']/math.log(10)/datafile['phot_rp_mean_flux'])

To make things easier, add the bp_rp and parallax_over_error features.

In [ ]:
# The color of the star
# In astronomy, a color is defined as the difference between magnitudes in different pass bands
datafile['bp_rp'] = datafile['phot_bp_mean_mag'] - datafile['phot_rp_mean_mag']

Select data with positive parallax value ($\omega>0$) and error of G magnitude ($\sigma_G$) $< 0.005$.

In [ ]:
data = datafile[(datafile['parallax'] > 0.) & (datafile['e_Gmag'] < 0.005)].reset_index(drop=True)

del datafile

## 2 PRIOR VISUALIZATION

### 2.1 VISUALIZATION: Spatial Distribution

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.plot(data['ra'], data['dec'], ',')

plt.xlabel(r'$\alpha$ (deg)')
plt.ylabel(r'$\delta$ (deg)')

plt.show()

### 2.2 VISUALIZATION: Vector Point Diagram

In [ ]:
# This is the velocity of stars
fig = plt.figure(figsize=(6, 6))
plt.plot(data['pmra'], data['pmdec'], ',')

plt.xlabel(r'$\mu_{\alpha*}$ (mas/yr)')
plt.ylabel(r'$\mu_{\delta}$ (mas/yr)')

# Stars in a cluster are expected to move in the same velocity so we focus on a small area
plt.xlim(-30,30)
plt.ylim(-30,30)

plt.show()

### 2.3 VISUALIZATION: Color Magnitude Diagram

In [ ]:
fig = plt.figure(figsize=(6, 8))
plt.plot(data['bp_rp'], data['phot_g_mean_mag'], ',')

ax = plt.gca()
ax.invert_yaxis()
plt.xlim(0., 3.)

plt.xlabel('bp - rp') # Color
plt.ylabel('g') # Magnitude

#This is a color-magnitude diagram (A type of HR Diagram)
plt.show()

## 3 HDBSCAN
Hierarchichal Density--Based Spatial Clustering of Applications with Noise (HDBSCAN) uses the density of points to determine the membership of a cluster.

### 3.1 HDBSCAN: Preparing the data
We are using the points on the **proper motion along the right ascension**, **proper motion along the declination** and the **parallax of each star** to determine their membership. Each star in a cluster is expected to have the same points for these three data.

In [ ]:
# We feed HDBSCAN to calculate the clustering based on these parameters
df = data[["pmra", "pmdec", "parallax"]] # pm = proper motion along those directions
df = df.to_numpy().astype("float32", copy = False)

# Normalizing the data
stscaler_df = StandardScaler().fit(df)
df_ = stscaler_df.transform(df)

### 3.2 HDBSCAN: Run HDBSCAN

In [ ]:
clus_size = 2 * df_.shape[1]

clusterer = hdbscan.HDBSCAN(clus_size)
cluster_labels = clusterer.fit_predict(df_)

data['hdbscan'] = cluster_labels

del df, df_, cluster_labels

### 3.3 HDBSCAN: Check
Check each clustering result. The color bar in the image shows the label of each cluster obtained.

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
gr = ax.scatter(data['pmra'], data['pmdec'], s=10, c=data['hdbscan'])#, edgecolor='')

fig.colorbar(gr, ax=ax)
ax = plt.gca()
ax.invert_yaxis()
plt.xlim(-30,30)
plt.ylim(-30,30)

plt.xlabel(r'$\mu_{\alpha*}$ (mas/yr)')
plt.ylabel(r'$\mu_{\delta}$ (mas/yr)')

plt.show()

Distribution of group labels of all data.

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(data['hdbscan'])

plt.xlabel('Label of Cluster')
plt.ylabel('Number of Sources')

plt.show()

The number of members from each clustering result.

In [ ]:
data['hdbscan'].value_counts()

### 3.4 HDBSCAN: Cleaning

Separate the data with a label that shows the background data (`label = -1`).

In [ ]:
result_hdbscan = data[data['hdbscan'] >= 0].reset_index(drop=True)

c = result_hdbscan['hdbscan'].value_counts()
del result_hdbscan

Find the cluster with the most number. Assuming the data used only consists of the background and one stellar cluster.

In [ ]:
n_max = c.index[np.argmax(c)]
del c

result = data[data['hdbscan'] == n_max]
result.to_csv(OUTPUTDIR + "results_" + CLUSTER + ".csv", index=False)

## 4 RESULTS VISUALIZATION

### 4.1 RESULTS: Spatial Distribution
The location of each stars in a cluster should not be far from each other.

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = plt.subplot()
plt.plot(data['ra'], data['dec'], '.', mec='silver', mfc='darkgray', markersize=1., label="All sources")
plt.plot(result['ra'], result['dec'], 'o', mfc='tab:orange', markersize=2., label="HDBSCAN")

plt.xlabel(r'$\alpha$ (deg)')
plt.ylabel(r'$\delta$ (deg)')
plt.legend()
plt.title("Spatial distribution of " + CLUSTER + " star cluster")
plt.savefig(OUTPUTDIR + FILENAME + "_spatial.png")
plt.show()

### 4.2 RESULTS: Vector Point Diagram
Stars in a cluster is expected to move in the same velocity

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.plot(data['pmra'], data['pmdec'], '.', mec='silver', mfc='darkgray', markersize=5., label="All sources")
plt.plot(result['pmra'], result['pmdec'], 'o', mfc='tab:orange', mec='None', markersize=5., label="HDBSCAN")

plt.xlabel(r'$\mu_{\alpha*}$ (mas/yr)')
plt.ylabel(r'$\mu_{\delta}$ (mas/yr)')

plt.xticks()
plt.yticks()

XMED = np.median(result['pmra'])
YMED = np.median(result['pmdec'])

XVAR = np.median(abs(result['pmra']-XMED))
YVAR = np.median(abs(result['pmdec']-YMED))

VAR = (XVAR + YVAR)/2

plt.xlim(XMED+6*VAR,XMED-6*VAR)
plt.ylim(YMED+6*VAR,YMED-6*VAR)

del XMED, XVAR, YMED, YVAR, VAR

plt.legend()
plt.title("Vector Point Diagram of " + CLUSTER + " star cluster")

plt.savefig(OUTPUTDIR + FILENAME + "_vector_point.png")
plt.show()

### 4.3 RESULTS: Color Magnitude Diagram
A cluster of stars is expected to follow the pattern from HR Diagram

In [ ]:
plt.figure(figsize=(6, 8))
plt.plot(data['bp_rp'], data['phot_g_mean_mag'], '.', mec='silver', mfc='darkgray', markersize=2., label="All sources")
plt.plot(result['bp_rp'], result['phot_g_mean_mag'], 'o', color='tab:orange', markersize=2., label=r"HDBSCAN")

plt.xlabel(r'$G_{BP}-G_{RP}$')
plt.ylabel(r'$G$ (mag)')

plt.xlim(0., 3.)
plt.gca().invert_yaxis()
plt.legend()
plt.title("Color Magnitude Diagram of "+ CLUSTER + " star cluster")

plt.savefig(OUTPUTDIR + FILENAME + "_color_magnitude.png")
plt.show()

### 4.4 RESULTS: Parallax Distribution
A cluster of stars is expected to be close together (i.e. have the same parallax)

In [ ]:
bins_all = np.arange(data['parallax'].min(), data['parallax'].max(), .01)
bins_sam = np.arange(result['parallax'].min(), result['parallax'].max(), .01)

In [ ]:
plt.figure(figsize=(6, 4))
data.parallax.hist(bins=bins_all, color='gray', label="All Sources")
result.parallax.hist(bins=bins_sam, color='orange', label="HDBSCAN")

plt.xlabel(r'$\omega$ (mas)')
plt.ylabel('Number of Sources')

plt.xlim(0, 5)

plt.xticks()
plt.yticks()

plt.legend()
plt.title("Parallax Distribution of " + CLUSTER + " star cluster")

plt.savefig(OUTPUTDIR + FILENAME + "_parallax_dist.png")
plt.show()

## 5 ADDITIONAL TASKS

### 5.1 Determine the center of the stellar cluster

In [ ]:
ra_c    = np.median(result['ra'])
dec_c   = np.median(result['dec'])
pmra_c  = np.median(result['pmra'])
pmdec_c = np.median(result['pmdec'])

fig = plt.figure(figsize=(6, 6))
ax = plt.subplot()
plt.plot(data['ra'], data['dec'], '.', mec='silver', mfc='darkgray', markersize=1., label="All sources")
plt.plot(result['ra'], result['dec'], 'o', mfc='tab:orange', markersize=2., label="HDBSCAN")
plt.plot(ra_c, dec_c, 'x')

plt.xlabel(r'$\alpha$ (deg)')
plt.ylabel(r'$\delta$ (deg)')
plt.legend()
plt.show()

### 5.2 The velocity of the center of cluster

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.plot(data['pmra'], data['pmdec'], '.', mec='silver', mfc='darkgray', markersize=5., label="All sources")
plt.plot(result['pmra'], result['pmdec'], 'o', mfc='tab:orange', mec='None', markersize=5., label="HDBSCAN")
plt.plot(pmra_c, pmdec_c, '+')

plt.xlabel(r'$\mu_{\alpha*}$ (mas/yr)')
plt.ylabel(r'$\mu_{\delta}$ (mas/yr)')

plt.xticks()
plt.yticks()

XMED = np.median(result['pmra'])
YMED = np.median(result['pmdec'])

XVAR = np.median(abs(result['pmra']-XMED))
YVAR = np.median(abs(result['pmdec']-YMED))

VAR = (XVAR + YVAR)/2

plt.xlim(XMED+6*VAR,XMED-6*VAR)
plt.ylim(YMED+6*VAR,YMED-6*VAR)

del XMED, XVAR, YMED, YVAR, VAR

plt.legend()
plt.show()

## 6. References
Below is a list of references used by the lecturer

1. Agarwal, Manan, et al. "ML-MOC: Machine Learning (kNN and GMM) based Membership determination for Open Clusters." Monthly Notices of the Royal Astronomical Society 502.2 (2021): 2582-2599.
2. Campello, Ricardo JGB, et al. "Hierarchical density estimates for data clustering, visualization, and outlier detection." ACM Transactions on Knowledge Discovery from Data (TKDD) 10.1 (2015): 1-51.
3. Chen, W. P., C. W. Chen, and C. G. Shu. "Morphology of Galactic open clusters." The Astronomical Journal 128.5 (2004): 2306.
4. Ester, Martin, et al. "A density-based algorithm for discovering clusters in large spatial databases with noise." kdd. Vol. 96. No. 34. 1996.
5. Gaia Collaboration. "VizieR Online Data Catalog: Gaia DR2 (Gaia Collaboration, 2018)." VizieR Online Data Catalog (2018): I-345.
6. Kounkel, Marina, and Kevin Covey. "Untangling the Galaxy. I. Local Structure and Star Formation History of the Milky Way." The Astronomical Journal 158.3 (2019): 122.
7. McInnes, Leland, John Healy, and Steve Astels. "hdbscan: Hierarchical density based clustering." Journal of Open Source Software 2.11 (2017): 205.
8. Pedregosa, Fabian, et al. "Scikit-learn: Machine learning in Python." the Journal of machine Learning research 12 (2011): 2825-2830.










